In [8]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load dataset
features = pd.read_csv("weatherAUS_cleaned.csv")
X = features.drop(["RainTomorrow"], axis=1).values
y = features["RainTomorrow"].values

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

# Define PyTorch ANN model
class ANNModel(nn.Module):
    def __init__(self, input_size, neurons_layer1, neurons_layer2, neurons_layer3, dropout_rate1, dropout_rate2):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, int(neurons_layer1))
        self.fc2 = nn.Linear(int(neurons_layer1), int(neurons_layer2))
        self.fc3 = nn.Linear(int(neurons_layer2), int(neurons_layer3))
        self.dropout1 = nn.Dropout(dropout_rate1)
        self.fc4 = nn.Linear(int(neurons_layer3), 8)
        self.dropout2 = nn.Dropout(dropout_rate2)
        self.fc5 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout2(x)
        x = self.sigmoid(self.fc5(x))
        return x

# Objective function for Simulated Annealing
def evaluate(params):
    neurons_layer1, neurons_layer2, neurons_layer3, dropout_rate1, dropout_rate2 = params
    
    neurons_layer1 = np.clip(neurons_layer1, 16, 128).astype(int)
    neurons_layer2 = np.clip(neurons_layer2, 16, 128).astype(int)
    neurons_layer3 = np.clip(neurons_layer3, 8, 64).astype(int)
    dropout_rate1 = np.clip(dropout_rate1, 0.1, 0.5)
    dropout_rate2 = np.clip(dropout_rate2, 0.1, 0.5)
    
    model = ANNModel(X_train.shape[1], neurons_layer1, neurons_layer2, neurons_layer3, dropout_rate1, dropout_rate2).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    
    model.train()
    for epoch in range(150):  # Reduced for efficiency
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        predictions = (predictions >= 0.5).float()
        accuracy = (predictions == y_test_tensor).float().mean().item()
    
    return -accuracy  # Minimize the negative accuracy

# Simulated Annealing (SA) function
def simulated_annealing(evaluate, bounds, max_iter=50, initial_temp=100, temp_decay=0.98):
    # Initialize parameters randomly within the bounds
    params = [random.uniform(bounds[i][0], bounds[i][1]) for i in range(len(bounds))]
    
    current_solution = params
    current_cost = evaluate(current_solution)
    best_solution = current_solution
    best_cost = current_cost
    
    temperature = initial_temp
    
    for iteration in range(max_iter):
        # Perturb the solution
        next_solution = current_solution.copy()
        idx = random.randint(0, len(bounds) - 1)
        next_solution[idx] = random.uniform(bounds[idx][0], bounds[idx][1])
        
        # Evaluate the new solution
        next_cost = evaluate(next_solution)
        
        # Accept the new solution with a probability based on the temperature
        if next_cost < current_cost or random.uniform(0, 1) < np.exp((current_cost - next_cost) / temperature):
            current_solution = next_solution
            current_cost = next_cost
            
            # Update the best solution found
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost
        
        # Decrease temperature
        temperature *= temp_decay
    
    return best_solution, best_cost

# Set bounds for the hyperparameters
bounds = [
    (16, 128),  # neurons_layer1
    (16, 128),  # neurons_layer2
    (8, 64),    # neurons_layer3
    (0.1, 0.5), # dropout_rate1
    (0.1, 0.5)  # dropout_rate2
]

# Run Simulated Annealing (SA)
best_params, best_cost = simulated_annealing(evaluate, bounds)

# Extract the best hyperparameters
best_neurons_layer1, best_neurons_layer2, best_neurons_layer3, best_dropout_rate1, best_dropout_rate2 = best_params

print(f"Best Hyperparameters:")
print(f"Neurons Layer 1: {int(best_neurons_layer1)}")
print(f"Neurons Layer 2: {int(best_neurons_layer2)}")
print(f"Neurons Layer 3: {int(best_neurons_layer3)}")
print(f"Dropout Rate 1: {best_dropout_rate1:.2f}")
print(f"Dropout Rate 2: {best_dropout_rate2:.2f}")


Using device: cuda
Best Hyperparameters:
Neurons Layer 1: 126
Neurons Layer 2: 115
Neurons Layer 3: 49
Dropout Rate 1: 0.11
Dropout Rate 2: 0.19
